In [2]:
import re
import json
from nltk import corpus as corpiss
from transformers import AutoTokenizer


/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
f = open('./jeopardy.json')
corpus = json.load(f)
out = []
for i,item in enumerate(corpus):
    # Remove any items with quesitons with links (e.g. <a href=")
    if bool(re.search('href',item['question'])):
        continue
    # Set values to ints. Final jeopardy questions have a value of "None"
    if item['value'] is not None:
        item['value'] = int(item['value'][1:].replace(',',''))    
    item['question'] = item['question'][1:-1]
    del item['air_date']
    del item['round']
    del item['show_number']
    out.append(item)
corpus = out
# print(corpus[0])
# [i['question'] for i in corpus[:100]]
corpus[38]['question']

"In 2003 this airline agreed to buy KLM, creating Europe's largest airline"

In [20]:
with open('jeopardy_parsed.json', 'w', encoding='utf-8') as f:
    json.dump(corpus, f, ensure_ascii=False, indent=4)